In [1]:
import sys

In [2]:
import os

In [3]:
from pathlib import Path

In [4]:
project_dir = Path(__name__).resolve().parents[1]

In [5]:
sys.path.insert(0, str(project_dir))

In [10]:
sys.path.insert(0, str(project_dir/'src'/'data'))

In [11]:
from scrape_bol import *

In [12]:
from get_products import *

In [13]:
from datetime import datetime

In [ ]:
s = SearchTerm(search_term='philips sonicare', search_domain = 'bol.com', max_pages = 1)

In [ ]:
len(s.productlist)

In [ ]:
print(s.Scraper.page)

In [ ]:
dir(s)

In [ ]:
datetime.now().strftime("%Y-%m-%d %I:%M%p")

In [ ]:
to_save_data = ['\t'.join([p.provider if p.provider else '', 
                          p.provider_id if p.provider_id else '', 
                           p.name  if p.name else '', 
                           p.price  if p.price else '', 
                           p.original_price  if p.original_price else '' ]) 
                for p in s.productlist]

In [ ]:
with open("../data/raw/price_tracker.tsv", 'a') as file:
    file.writelines(to_save_data)

In [213]:
bol = BolScraper('https://www.bol.com/nl/nl/s/?searchtext=brita')

In [221]:
pl_bs4 = bol.page.find_all('li', attrs = {'class': 'product-item--row'})

In [228]:
example = [p for p in pl_bs4 if p.attrs['data-id'] == '9200000074016762'][0]

In [ ]:
price = 

In [237]:
example.find('span', attrs = {'data-test': 'price'}).contents[0].strip()

'42'

In [238]:
example.find('sup', attrs = {'data-test': 'price-fraction'}).contents[0].strip()

'94'

In [ ]:
pl = bol.get_productlist()

In [ ]:
bol = BolScraper('https://www.bol.com/nl/nl/s/?page=8&searchtext=philips+sonicare&view=list') # No products

In [ ]:
pl = bol.get_productlist()

In [ ]:
pl

In [ ]:
# pl

In [ ]:
from pathlib import Path

In [ ]:
project_dir = Path(__name__).resolve().parents[2]

In [ ]:
project_dir/'data'/'raw'/'example.csv'

# Create DB

In [ ]:
import sqlite3
from sqlite3 import Error

In [ ]:
conn = sqlite3.connect("../data/raw/price_tracker.db",  timeout=10)

In [ ]:
sql_create_products_table = """ CREATE TABLE IF NOT EXISTS projects (
                                    id integer PRIMARY KEY,
                                    name text NOT NULL,
                                    begin_date text,
                                    end_date text
                                ); """

sql_create_prices_table = """CREATE TABLE IF NOT EXISTS prices (
                                datetime datetime NOT NULL,
                                provider text ,
                                provider_id text,
                                name text,
                                price numeric,
                                original_price numeric
                            );"""

In [ ]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [ ]:
create_table(conn, sql_create_prices_table)

In [ ]:
conn.close()

# Pandas

In [260]:
import pandas as pd

data_dir = project_dir/'data'/'external'
max_file = data_dir /'new_price.tsv'
second_max_file = data_dir / 'old_price.tsv'

df_max = pd.read_csv(max_file, sep = '\t')
df_second_max = pd.read_csv(second_max_file, sep = '\t')

df = pd.merge(df_second_max, df_max, left_on = ['provider_id'], right_on = ['provider_id'], 
        suffixes = (None, '_old'))

df['discount'] = 1 - df['price'] / df['price_old'].str.strip('-').astype(float)

df_notify = df[df['discount'] >=0.05]

cols = ['provider', 'name', 'price', 'price_old', 'URL']

if len(df_notify)>0:
    records = df_notify[cols].to_records(index = False)

In [262]:
record = records[0]

In [263]:
records = df_notify.to_dict(orient = 'records')
r = []
for record in records:
    price = record['price']
    original_price = record['price_old']
    url = record['URL']

    r.append('\n'.join([record['provider'], record['name'], f'{price}~{original_price}~',
                            f'<{url}|Product Link>'
                           ])
            )

In [264]:
'\n\n'.join(r)

''

# Pandas All Files

In [173]:
from concurrent.futures import ProcessPoolExecutor
import pandas as pd

In [160]:
data_dir = project_dir/'data'/'raw'

In [170]:
example_file = list(data_dir.iterdir())[0]

In [189]:
example_file.name

'2021-10-05 12:00PM.tsv'

In [194]:
def read_each_tsv(file):
    try:
        df = pd.read_csv(file, sep = '\t', )
        df['parse_date'] = file.name.rstrip('.tsv')
        return df
    except:
        print(file)

In [239]:
with ProcessPoolExecutor() as pool:
    results = pool.map(read_each_tsv, data_dir.iterdir())

/home/jetson/pi_projects/raspberrypi_projects/price_tracker/data/raw/.gitkeep


In [240]:
df = pd.concat(results)

In [241]:
df['parse_date'] = pd.to_datetime(df['parse_date'])

In [252]:
df_brita = df[df['name'].str.contains('BRITA Waterfilterkan Style Cool Blue')].copy()

In [253]:
df_brita.set_index('parse_date', inplace = True)

In [254]:
df_brita.sort_index(inplace = True)

In [255]:
df_brita.tail()

,provider,provider_id,name,price,original_price,URL
parse_date,,,,,,
2021-10-06 05:02:00,bol,9200000074016762,"BRITA Waterfilterkan Style Cool Blue 2,4L + 3 ...",42.94,NaN,https://bol.com/nl/nl/p/brita-waterfilterkan-s...
2021-10-06 06:02:00,bol,9200000074016762,"BRITA Waterfilterkan Style Cool Blue 2,4L + 3 ...",42.94,NaN,https://bol.com/nl/nl/p/brita-waterfilterkan-s...
2021-10-06 07:02:00,bol,9200000074016762,"BRITA Waterfilterkan Style Cool Blue 2,4L + 3 ...",NaN,NaN,https://bol.com/nl/nl/p/brita-waterfilterkan-s...
2021-10-06 08:02:00,bol,9200000074016762,"BRITA Waterfilterkan Style Cool Blue 2,4L + 3 ...",42.94,NaN,https://bol.com/nl/nl/p/brita-waterfilterkan-s...
2021-10-06 08:34:00,bol,9200000074016762,"BRITA Waterfilterkan Style Cool Blue 2,4L + 3 ...",42.94,NaN,https://bol.com/nl/nl/p/brita-waterfilterkan-s...


# Dotenv

In [ ]:
from dotenv import find_dotenv, load_dotenv

In [ ]:
load_dotenv(find_dotenv())

In [ ]:
f = project_dir.parent / 'Utils'

In [ ]:
f

# Selenium Driver

In [14]:
bol = BolScraper('https://www.bol.com/nl/nl/s/?searchtext=brita', driver_method = 'selenium')

In [15]:
pl = bol.get_productlist()

In [20]:
bol.driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="366ffb8fb0c52d2d66a149f2cd27670c")>

In [17]:
# pl

In [18]:
bol.close()